<h1 align="center">Clustering Covid-19 Cases in Toronto - Canada</h1>

<p align="justify">In this notebook we are going to cluster the data related to the Covid-19 cases in the city of Toronto, Canada. To accomplish this, we must download the data sources from the following URLs: <a href="https://open.toronto.ca/dataset/covid-19-cases-in-toronto/">Toronto Covid Data</a> and <a href="https://cocl.us/Geospatial_data">Toronto Postal Code Coordinates</a>.Of course it will be necessary to clean and normalize the data to be able to get the visual insight that we are looking for in the project. The main goal is to see the cluster distribution if the Covid-19 cases in the city of Toronto at the date of the data downloaded, with this, the people arriving to Toronto can be informed about the virus hot spots and, they will avoid the venues in the neighborhoods with most active cases of this disease.</p>

<h2>1. Managing the Data</h2>

<h3>1.1 Downloading the data</h3>
<p>We must obtain the csv files to get the data for our insights. To do this, we must import the <b>Pandas</b> and <b>Wget</b> libraries</p>

In [1]:
import pandas as pd
import wget
print("Libraries imported!")

Libraries imported!


<p>Proceeding to download the data from their respective sources. Both files will be saved in the project folder</p>

In [9]:
wget.download("https://ckan0.cf.opendata.inter.prod-toronto.ca/download_resource/e5bf35bc-e681-43da-b2ce-0242d00922ad?format=csv")
wget.download("https://cocl.us/Geospatial_data")
print("Csv files downloaded!")

Csv files downloaded!


<h3>1.2 Transforming the data</h3>
<p>With the csv files in our project folder, we have to convert those files into pandas dataframe. Of course, pandas has the tools to do this task</p>

In [10]:
covid_df = pd.read_csv('COVID19 cases.csv')
coordinates_df = pd.read_csv('Geospatial_Coordinates.csv')
print("Dataframe conversion done")

Dataframe conversion done


<p></p>Let's check our dataframes

In [11]:
# Toronto Covid-19 dataframe
covid_df.head()

,_id,Assigned_ID,Outbreak Associated,Age Group,Neighbourhood Name,FSA,Source of Infection,Classification,Episode Date,Reported Date,Client Gender,Outcome,Currently Hospitalized,Currently in ICU,Currently Intubated,Ever Hospitalized,Ever in ICU,Ever Intubated
0,1,1,Sporadic,50 to 59 Years,Willowdale East,M2N,Travel,CONFIRMED,2020-01-22,2020-01-23,FEMALE,RESOLVED,No,No,No,No,No,No
1,2,2,Sporadic,50 to 59 Years,Willowdale East,M2N,Travel,CONFIRMED,2020-01-21,2020-01-23,MALE,RESOLVED,No,No,No,Yes,No,No
2,3,3,Sporadic,20 to 29 Years,Parkwoods-Donalda,M3A,Travel,CONFIRMED,2020-02-05,2020-02-21,FEMALE,RESOLVED,No,No,No,No,No,No
3,4,4,Sporadic,60 to 69 Years,Church-Yonge Corridor,M4W,Travel,CONFIRMED,2020-02-16,2020-02-25,FEMALE,RESOLVED,No,No,No,No,No,No
4,5,5,Sporadic,60 to 69 Years,Church-Yonge Corridor,M4W,Travel,CONFIRMED,2020-02-20,2020-02-26,MALE,RESOLVED,No,No,No,No,No,No


In [12]:
# Toronto Coordinates Dataframe
coordinates_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


<h3>1.3 Cleaning and normalizing the data</h3>
<p align="justify">Observing the dataframes is obvious that it will be necessary to clean and normalize the Toronto Covid dataframes because, we will not need all the info there. For our project we only need the following features: Neighborhood name, FSA(Postal Code), Classification and Outcome. But first we need to remove the space in the Postal Code column in the Toronto Coordinates dataframe</p>

In [13]:
coordinates_df.rename(columns={'Postal Code': 'Postal_Code'}, inplace=True)
coordinates_df.head()

,Postal_Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
# Drop the columns in the Covid-19 dataframe that are not necessary for our project
covid_df.drop(columns=[ '_id', 'Assigned_ID', 'Outbreak Associated', 'Age Group', 'Source of Infection', 'Episode Date', 'Reported Date', 'Client Gender', 'Currently Hospitalized', 'Currently in ICU', 'Currently Intubated', 'Ever Hospitalized', 'Ever in ICU', 'Ever Intubated'], inplace=True)
covid_df.head()

,Neighbourhood Name,FSA,Classification,Outcome
0,Willowdale East,M2N,CONFIRMED,RESOLVED
1,Willowdale East,M2N,CONFIRMED,RESOLVED
2,Parkwoods-Donalda,M3A,CONFIRMED,RESOLVED
3,Church-Yonge Corridor,M4W,CONFIRMED,RESOLVED
4,Church-Yonge Corridor,M4W,CONFIRMED,RESOLVED


<p>We already have the data that we will use, but lets rename some of the columns to names with more sense and order the columns position for a better view</p>

In [15]:
# Let's rename the columns Neighbourhood Name, Classification and FSA
covid_df.rename(columns={'Neighbourhood Name': 'Neighborhood', 'FSA': 'Postal_Code', 'Classification': 'Status'}, inplace=True)

# Reordering the columns positions

covid_reduced_df = covid_df[['Postal_Code', 'Neighborhood', 'Status', 'Outcome']]
covid_reduced_df.head()

,Postal_Code,Neighborhood,Status,Outcome
0,M2N,Willowdale East,CONFIRMED,RESOLVED
1,M2N,Willowdale East,CONFIRMED,RESOLVED
2,M3A,Parkwoods-Donalda,CONFIRMED,RESOLVED
3,M4W,Church-Yonge Corridor,CONFIRMED,RESOLVED
4,M4W,Church-Yonge Corridor,CONFIRMED,RESOLVED


<p>Now, we are going to clean the data, we will follow the next instructions:<br>
<ul>
<li>Drop the nan/null values in Postal Code, because without this data we cannot map the Neighborhood</li>
<li>The nan/null values in the Neighborhood column will be replaced by the Postal Code value</li>
<li>In the Status column, we only need the current confirmed cases</li>
<li>In the Outcome column, we only need the current active cases</li>
</ul>
</p>

In [16]:
# Before the changes let's check the dataframe shape
covid_reduced_df.shape

(15338, 4)

In [17]:
# Drop the nan/null values in the Postal Code column
covid_reduced_df.dropna(subset=['Postal_Code'], inplace=True)
covid_reduced_df.shape

(14775, 4)

In [18]:
# Fill the nan/null values in Neigborhood with the Postal Code. First, we must now how many records in Neighborhood without data we have in the dataframe
count = covid_reduced_df["Neighborhood"].isna().sum()
print(count)

46


In [19]:
# Lets replace the null data in Neighborhoods
covid_reduced_df.Neighborhood.fillna(covid_reduced_df.Postal_Code, inplace=True)

# Checking if there are still some null rows
count = covid_reduced_df["Neighborhood"].isna().sum()
print(count)

0


In [20]:
# Removing values that are not necessary in the column Status

covid_clean_df = covid_reduced_df[covid_reduced_df.Status == 'CONFIRMED']
covid_clean_df.shape

(13673, 4)

In [21]:
# Check if we have rows in Status with other values than CONFIRMED

covid_clean_df.groupby(by='Status').agg('count')

,Postal_Code,Neighborhood,Outcome
Status,,,
CONFIRMED,13673,13673,13673


In [22]:
# Removing values that are not necessary in the column Outcome

covid_clean_df = covid_clean_df[covid_clean_df.Outcome == 'ACTIVE']
covid_clean_df.shape

(349, 4)

In [23]:
# Check if we have rows in Outcome with other values than ACTIVE

covid_clean_df.groupby(by='Outcome').agg('count')

,Postal_Code,Neighborhood,Status
Outcome,,,
ACTIVE,349,349,349


<p>Ok, we finished this data cleaning, we start with a dataframe with 14911 records and was filtered until get 532 records, lets go with the final part of the data managing</p>

<h3>1.4 Merging the dataframes</h3>
<p>With both dataframes ready, we will proceed to merge the Latitude and Longitude from the coordinates_df to the covid_clean_df to get a new dataframe called covid_toronto_df</p>

In [24]:
# We need to reset the index in the covid_clean_df

covid_clean_df.reset_index(inplace=True, drop=True)

# Merge the Latitude and Longitude postal code values

covid_toronto_df = pd.merge(covid_clean_df, coordinates_df, on='Postal_Code')
covid_toronto_df.head()

,Postal_Code,Neighborhood,Status,Outcome,Latitude,Longitude
0,M8V,Mimico (includes Humber Bay Shores),CONFIRMED,ACTIVE,43.605647,-79.501321
1,M8V,New Toronto,CONFIRMED,ACTIVE,43.605647,-79.501321
2,M8V,New Toronto,CONFIRMED,ACTIVE,43.605647,-79.501321
3,M8V,Mimico (includes Humber Bay Shores),CONFIRMED,ACTIVE,43.605647,-79.501321
4,M1B,Rouge,CONFIRMED,ACTIVE,43.806686,-79.194353


In [25]:
# Check the dataframe shape
covid_toronto_df.shape

(349, 6)

<h2>2. Maping and Clustering the Data</h2>

<h3>2.1 Maping the data</h3>
<p>Using the previous dataframe, lets map the Covid-19 cases in Toronto. First, we need to import the libraries and modules to accomplish the task</p>

In [26]:
from geopy.geocoders import Nominatim
import folium
import requests
from folium.plugins import FastMarkerCluster
print('Libraries imported.')

Libraries imported.


<p>We will need to save the Toronto coordinates in their respective variables</p>

In [27]:
address = 'Toronto, ON'
geolocator = Nominatim(user_agent="toronto_mapping")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


<p>Generating Toronto's map</p>

In [99]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

FastMarkerCluster(data=list(zip(covid_toronto_df['Latitude'].values, covid_toronto_df['Longitude'].values)), name='Clustering').add_to(map_toronto)
folium.LayerControl(position='topright').add_to(map_toronto)

for lat, lng, label in zip(covid_toronto_df['Latitude'], covid_toronto_df['Longitude'], covid_toronto_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.vector_layers.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='grey',
        fill=False,
        fill_opacity=0.5).add_to(map_toronto) 

map_toronto

In [89]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(covid_toronto_df['Latitude'], covid_toronto_df['Longitude'], covid_toronto_df['Neighborhood']):
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color=rainbow,
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

NameError: name 'rainbow' is not defined